#### all imports

In [1]:
import tensorflow as tf
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.applications.vgg19 import VGG19 , preprocess_input
from keras.utils import load_img, img_to_array
from tqdm import tqdm
import re
import pickle


2024-06-26 05:52:07.636846: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-26 05:52:07.639848: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-26 05:52:07.677904: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-26 05:52:07.677937: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-26 05:52:07.679146: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
import json #to create dataset metadata json file

### initialize a generator for the validation images 
#### Note: change train2014 for the train images

In [3]:
# Define paths to dataset and output files
data_dir = "datasets/"
output_dir = "VGG19"
image_dir = os.path.join(data_dir, "training")
output_file = os.path.join(output_dir, "val_features.npy")

# Define a data generator to preprocess the images
target_size = (224, 224)
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
generator = datagen.flow_from_directory(
    image_dir,
    target_size=target_size,
    batch_size=32,
    class_mode=None,
    shuffle=False
)

Found 82783 images belonging to 1 classes.


#### load VGG19 and extract images features

In [4]:
# Create an VGG19 model to extract image features
base_model = VGG19(weights='imagenet')

In [5]:
model = Model(inputs=base_model.input, outputs=base_model.get_layer('flatten').output)

In [6]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import sys
from PIL import Image
sys.modules['Image'] = Image
from PIL import Image
print(Image.__file__)

/home/senju/vaq/env/lib/python3.11/site-packages/PIL/Image.py


In [7]:
from PIL import Image
import scipy


# Extract image features for each image in the training set
train_features = []
for i in tqdm(range(len(generator))):
    batch = generator.next()
    features = model.predict_on_batch(batch)
    train_features.append(features)

# Concatenate and reshape the extracted features into a numpy array
train_features = np.concatenate(train_features)
train_features = train_features.reshape((len(generator.filenames), -1))


100%|██████████| 2587/2587 [1:28:11<00:00,  2.05s/it]


#### save features with IDs in a dictionary in a pkl file

In [8]:
# add ids to features
img_ids = np.array([int(re.search("[0-9][0-9][0-9][0-9][0-9]+", gen).group()) for gen in generator.filenames])
image_features = {}
for i in range(len(img_ids)):
    image_features[img_ids[i]]= train_features[i]

In [9]:
# save dictionary to test_image_features.pkl file
with open('vgg19_train_data.pkl', 'wb') as fp:
    pickle.dump(image_features, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


# end